
# NoDiffTransformer & TripletNoDiffTransformer — Tutorial

This notebook demonstrates how to use the cleaned, **separate-file** modules:

- `nodiff_transformer.py` → `NoDiffTransformer`
- `triplet_nodiff_transformer.py` → `TripletNoDiffTransformer`
- `diff_transformer_utils.py` → shared utilities

**Input spec for this tutorial:** single-channel images of size **51×51** (i.e., tensors shaped `(N, 1, 51, 51)`).  


In [1]:

# If running in a fresh environment, uncomment and run:
# !pip install torch einops timm


In [1]:

import sys, os, torch

# Make sure the module directory is discoverable (adjust if needed)
sys.path.append("/global/u1/i/inadas/projects/publish/nodiff/nodiff/")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

## Import models

In [2]:

from nodiff_transformer import NoDiffTransformer
from triplet_nodiff_transformer import TripletNoDiffTransformer


## Create dummy inputs (channels=1, size=51×51)

In [3]:

# Single model input
x = torch.randn(2, 1, 51, 51, device=device)

# Triplet model inputs
xa = torch.randn(2, 1, 51, 51, device=device)
xp = torch.randn(2, 1, 51, 51, device=device)
xn = torch.randn(2, 1, 51, 51, device=device)

x.shape, xa.shape, xp.shape, xn.shape


(torch.Size([2, 1, 51, 51]),
 torch.Size([2, 1, 51, 51]),
 torch.Size([2, 1, 51, 51]),
 torch.Size([2, 1, 51, 51]))

## Instantiate models 

The model in the paper requires you to pass the device as an argument for training.

I am keeping it for the sake of consistency.

For real work, it is better practice to use register_buffer so that there is no device mismatch.

In [4]:

model = NoDiffTransformer((51,51), device=device).to(device)
triplet_model = TripletNoDiffTransformer((51,51), device=device).to(device)

def count_params(m):
    return sum(p.numel() for p in m.parameters())

count_params(model), count_params(triplet_model)


/global/homes/i/inadas/miniconda3/envs/realbogus/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


(65923243, 67802347)

## Forward pass

In [5]:

model.eval()
with torch.no_grad():
    out_single = model(x,x)  # If your model returns multiple items, adapt handling below.

type(out_single), getattr(out_single, "shape", None)


(torch.Tensor, torch.Size([2]))

In [6]:

triplet_model.eval()
with torch.no_grad():
    out_triplet = triplet_model(xa, xp, xn)

type(out_triplet), getattr(out_triplet, "shape", None)


(torch.Tensor, torch.Size([2]))